In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import numpy as np
import os, csv
from PIL import Image
import cv2

In [ ]:
#RUN THIS CELL TO IMPORT G_DRIVE
#Share dataset to your drive first.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir(r'/content/drive/MyDrive/ES_FaceMatch_Dataset')
print(os.getcwd())

train = csv.reader(open('train.csv'))
train_data = []

for line in train:
    try:
        train_data.append([line[0], line[1], int(line[2])])
    except:
        pass
print(train_data[2])

/content/drive/.shortcut-targets-by-id/1jVtyGeVkD0wuaIwqQ0-8rkJz8KDKIeOk/ES_FaceMatch_Dataset
['19865480214299.jpg', '55378374591770.jpg', 1]


In [ ]:
img = np.asarray(Image.open(r'dataset_images/'+train_data[2][0]))
print(img.shape, type(img))

(256, 256, 3) <class 'numpy.ndarray'>


In [ ]:
train_input1 = []
train_input2 = []
train_output = []
count = 0
img1 = None
img2 = None
for i in train_data:
    img1 = cv2.imread(r'dataset_images/'+i[0],cv2.IMREAD_COLOR)
    img1 = cv2.resize(img1,(256,256))
    #train_input1.append(np.asarray(Image.open(r'dataset_images/'+i[0])))
    train_input1.append(img1)
    
    img2 = cv2.imread(r'dataset_images/'+i[1],cv2.IMREAD_COLOR)
    img2 = cv2.resize(img2,(256,256))
    #train_input2.append(np.asarray(Image.open(r'dataset_images/'+i[1])))
    train_input2.append(img2)
    train_output.append(i[2])
    count += 1
    if count >= 5000:
      break

In [ ]:
train_input1_model = np.array(train_input1[0:3500])
train_input2_model = np.array(train_input2[0:3500])
output_train = np.array(train_output[0:3500])
val_input1_model = np.array(train_input1[3500:5000])
val_input2_model = np.array(train_input2[3500:5000])
output_val = np.array(train_output[3500:5000])
print(len(train_input1_model))
print(len(output_val))

3500
1500


In [ ]:
input1 = keras.layers.Input((256,256,3,)) #change 256 to the image dimension obtained after preprocessing. Should we do grayscale
input2 = keras.layers.Input((256,256,3,))
input = concatenate([input1, input2],axis=-1)
input.shape

print(train_input1_model[0].shape, len(train_input2_model), len(output_train))

(256, 256, 3) 3500 3500


In [ ]:
conv1 =  tf.keras.layers.Conv2D(filters=64, kernel_size=(7,7), strides=(2,2), padding='same',activation='relu', activity_regularizer=regularizers.l2(0.01))(input) #added some convolutions and dense layers hoping to get good representations
pool1 =  tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(conv1)                                         
bn =  tf.keras.layers.BatchNormalization()(pool1)                                                                                          
conv2 =  tf.keras.layers.Conv2D(filters=64, kernel_size=(1,1), strides=(1,1), padding='same',activation='relu', activity_regularizer=regularizers.l2(0.01))(bn)  
conv3 =  tf.keras.layers.Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same',activation='relu', activity_regularizer=regularizers.l2(0.01))(conv2) 
bn2 =  tf.keras.layers.BatchNormalization()(conv3)                                                                    
pool2 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(bn2)
conv4 = tf.keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), padding='same',activation='relu', activity_regularizer=regularizers.l2(0.01))(pool2) # new
conv5 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same',activation='relu', activity_regularizer=regularizers.l2(0.01))(conv4)
conv6 = tf.keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), padding='same',activation='relu', activity_regularizer=regularizers.l2(0.01))(conv5)
conv7 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same',activation='relu', activity_regularizer=regularizers.l2(0.01)),
pool3 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same', activity_regularizer=regularizers.l2(0.01))(conv6)                                    
flat =  tf.keras.layers.Flatten()(pool3)
dense1 =  tf.keras.layers.Dense(128,activation='relu', activity_regularizer=regularizers.l2(0.01))(flat)
norm1 = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(dense1) # L2 normalize embeddings
dense2 =  tf.keras.layers.Dense(128,activation='relu', activity_regularizer=regularizers.l2(0.01))(norm1)
dense3 = tf.keras.layers.Dense(32,activation='relu' , activity_regularizer=regularizers.l2(0.01))(dense2)
output = tf.keras.layers.Dense(1,activation = 'sigmoid', activity_regularizer=regularizers.l2(0.01))(dense3) #has to be 0 or 1




In [ ]:
model = Model([input1, input2], output)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 256, 256, 6)  0           input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 128, 128, 64) 18880       concatenate_2[0][0]        

In [ ]:
#train the model here
model.compile(optimizer=Adam(learning_rate=1e-3), loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])



In [ ]:
model.fit(
    x = [train_input1_model, train_input2_model],
    #x = train_input1_model,
    y = output_train,
    batch_size = 200,
    epochs = 10,
    verbose = 1,
    validation_data = ([val_input1_model, val_input2_model], output_val),
    #validation_data = train_input2_model,
    shuffle = True, # doesn't matter if we have only one epoch or no batches,
)

Epoch 1/10
18/18 [==============================] - 1086s 60s/step - loss: 1686069.2500 - binary_accuracy: 0.6174 - val_loss: 104.9500 - val_binary_accuracy: 0.6260
Epoch 2/10
18/18 [==============================] - 1080s 60s/step - loss: 106.4931 - binary_accuracy: 0.6234 - val_loss: 86.2394 - val_binary_accuracy: 0.6260
Epoch 3/10
